In [1]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
import torch

checkpoints = r"shenzhi-wang/Llama3.1-8B-Chinese-Chat"
cache_dir = r"./"
# model = AutoModelForCausalLM.from_pretrained(checkpoints,cache_dir=cache_dir) # 运行一次下载了就好了

from modelscope import snapshot_download
model_dir = snapshot_download('LLM-Research/Meta-Llama-3.1-8B-Instruct', cache_dir='./llama3.1_8b_chinese', revision='master')

In [2]:
from datasets import load_from_disk

datasets_dir = r"./Roleplay-anime"
ds = load_from_disk(datasets_dir)

In [3]:
ds['train'][3]

{'input': '你快走吧!看到你就烦',
 'output': '(我皱起眉头，心里有些难过。我知道他们可能对我有误解，但我也不想让他们讨厌我。我鼓起勇气，尽量让自己的语气听起来友好一些。) 喂，你们别这样嘛！我们都是同学，应该互相帮助才对啊。有什么误会我们可以坐下来好好谈谈，对吧？(我微笑着看着他们，希望他们能理解我的一片好心。)',
 'instruction': '久沼纱友（日语：久沼さゆ）是P.A.WORKS与漫画杂志《电击大王》合作的TV动画《来自风平浪静的明天》及其衍生作品的登场角色。\n久沼纱友是美滨小学和美滨中学的学生，与挚友潮留美海组成萝莉组。她是一个坚强、自立的女孩，为朋友两肋插刀的元气型早熟萝莉。在故事开始前的两年，她在学校里是被校园暴力的对象，直到美海出现，转移了周围同学的炮火。她对男主角先岛光恶作剧后与其成为欢喜冤家，喜欢叫光“章鱼”，相对的总是被光扯脸蛋。在为挚友美海打抱不平而破坏贡女人偶后，被要教育，受他的温柔影响对他产生好感，从此一直注视着他。在要冬眠的5年期间，她一直保持着对他的感情不改变，更为了能和他相配而努力学习，成为一直保持成绩全班第一的学霸。在要醒来后，她一直为自己注视着的要只将注意力放在千咲身上而挣扎。\n久沼纱友在故事中扮演着重要的角色，她在剧情中的重要性大幅度提升。她的登场时间很早，从第一话开始就出现，大部分时间都保持活跃。她和美海一起组成的萝莉组在剧中承担了很多重要的剧情，特别是在后半部分，她的角色发展更为复杂，也有更多的戏份。她在后半部分的表现赢得了很多观众的喜爱，被称为“纱友小天使”。\n久沼纱友在剧中的故事发展中经历了许多起伏。在五年前，她一直默默地喜欢着要，被他的冬眠打破了心中的平静。在五年后，她独自一人在大城市生活，努力成为一个女强人。然而，她内心深处对要的感情始终未曾改变。在剧中的一次偶遇中，她终于向要表白了她一直以来的心意。这一场表白让她释放了积压已久的情感，也让要意识到在陆地上还有人在等待着他、在他醒来的这些天里一直挂念着他。在剧情的发展中，久沼纱友扮演着一个坚定、努力追求自己幸福的女孩的角色。\n久沼纱友的形象也有一些特点，她经常扎着双马尾，给人一种可爱的感觉。她的声优是石原夏织，通过她的声音，为角色赋予了更多的个性和魅力。\n总之，久沼纱友是《来自风平浪静的明天》中的一个重要角色，她的故事发

In [4]:
def get_dataset(ds):
     # 处理数据集
    ds = ds['train']
    tokenizer = AutoTokenizer.from_pretrained(r'./tokenizer-fold/',
                                              use_fast=False, trust_remote_code=True)
    def process_func(example):
        
        MAX_LENGTH = 1024  # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
        input_ids, attention_mask, labels = [], [], []
        
        instruction = tokenizer(
            f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
            
            You should engage in conversation with the user according to the character's profile. Here are the character's name and details:
            {example['instruction']}<|eot_id|><|start_header_id|>user<|end_header_id|>
            
            {example['input']}<|eot_id|> <|start_header_id|>assistant<|end_header_id|>\n\n""",
            add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
        response = tokenizer(f"{example['output']}<|eot_id|>", add_special_tokens=False)
 
        input_ids = instruction["input_ids"] + response["input_ids"] + [128001]
        attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
        labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [128001]
        if len(input_ids) > MAX_LENGTH:  # 做一个截断
            input_ids = input_ids[:MAX_LENGTH]
            attention_mask = attention_mask[:MAX_LENGTH]
            labels = labels[:MAX_LENGTH]
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }
    dataset = ds.map(process_func, remove_columns=ds.column_names)
 
    return dataset,tokenizer

In [5]:
datas_test,tokenizer_test = get_dataset(ds)

In [6]:
print(tokenizer_test.decode(datas_test[12]["input_ids"]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
            
            You should engage in conversation with the user according to the character's profile. Here are the character's name and details:
            彩濑逢樱是《创作少女的恋爱公式》中的角色，她喜欢打棒球，曾经和男主是青梅竹马。尽管小时候她被别人故意说和男主有绯闻，但两人并没有表达彼此的心意。在才华中学重新相遇后，她展现了认真坦率的性格，非常可靠，并经常扮演吐槽角色。她是所有主要角色中知识最丰富的人，经常给其他角色提供帮助和建议。此外，彩濑逢樱是一位获得多个小说奖项的恋爱小说作家，享有很高的知名度。她喜欢冰淇淋和棒球，并且喜欢作品有大团圆结局。然而，尽管她在写作方面非常有才华并且有着丰富的知识，但她对御宅族的内容却不太了解，在与其他角色的对话中经常会产生误解或不同的解释。<|eot_id|><|start_header_id|>user<|end_header_id|>
            
            我请求加你为好友了<|eot_id|> <|start_header_id|>assistant<|end_header_id|>

（微笑着点头）好的呀，很高兴能和你成为朋友！（拿出手机，输入账号密码，添加对方为好友）以后有什么事情需要帮忙的话，尽管找我哦。（眨眼睛，露出一个友好的笑容）<|eot_id|><|end_of_text|>


In [7]:
import json
from transformers import TrainingArguments, Trainer, GenerationConfig, DataCollatorForSeq2Seq
from peft import LoraConfig, TaskType, get_peft_model
def get_train(model,datas,tokenizer):
    #peft的lora参数
    config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        inference_mode=False,  # 训练模式
        r=8,  # Lora 秩
        lora_alpha=32,  # Lora alaph，具体作用参见 Lora 原理
        lora_dropout=0.1  # Dropout 比例
    )
 
    peft_model = get_peft_model(model, config)
    print(peft_model.print_trainable_parameters())
 
    #训练的参数
    args = TrainingArguments(
        output_dir="./output/llama3.1",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        logging_steps=10,
        num_train_epochs=3,
        save_steps=100,
        learning_rate=1e-4,
        save_on_each_node=True,
        gradient_checkpointing=True
    )
 
    #开始训练
    trainer = Trainer(
        model=peft_model,
        args=args,
        train_dataset=datas,
        data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    )
    trainer.train()
    #保存模型
    peft_model_id="./llama3.1_lora"
    trainer.model.save_pretrained(peft_model_id)
    tokenizer.save_pretrained(peft_model_id)

In [8]:
datas_test

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 7593
})

In [9]:
def get_model():
    model = AutoModelForCausalLM.from_pretrained('./llama3.1_8b_chinese/LLM-Research/Meta-Llama-3___1-8B-Instruct', device_map="auto",torch_dtype=torch.bfloat16)
    print(model.device)
    model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法
    return model

In [10]:
def main():
    model=get_model()
    print(model.dtype)
    get_train(model,datas_test,tokenizer_test)
 
 
if __name__ == '__main__':
    main()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

cuda:0
torch.bfloat16
trainable params: 20,971,520 || all params: 8,051,232,768 || trainable%: 0.2605
None
[2024-10-05 11:11:34,463] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.1
 [WARNING]  using untested triton version (2.1.0), only 1.0.0 is known to be compatible


/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.850000
20,1.437700
30,1.407400
40,1.346900
50,1.296800
60,1.296800
70,1.318600
80,1.266800
90,1.290100
100,1.268000


/root/miniconda3/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in ./llama3.1_8b_chinese/LLM-Research/Meta-Llama-3___1-8B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in ./llama3.1_8b_chinese/LLM-Research/Meta-Llama-3___1-8B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/lib/pyth